In [1]:
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
RANDOM_SEED = 27

In [3]:
df_orig = pd.read_csv('./winequalityN.csv')
df_orig.dropna(inplace=True)
# df_white = df[df.type=='white'].drop(columns='type')
# df_red = df[df.type=='white'].drop(columns='type')

* highly imbalanced

In [4]:
from sklearn.preprocessing import StandardScaler

## scorers

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_fscore_support

In [44]:
df = df_orig.copy()
df['type'] = df.type.map({'red': 1, 'white': 0})
df['quality'] = df['quality'].map({3:0, 4:0, 5:1, 6:2, 7:3, 8:4, 9:4})
x_train_val, x_test, y_train_val, y_test = train_test_split(df.drop(columns=['quality']),
                                                             df['quality'],
                                                             stratify=df['quality'],
                                                             test_size=0.2,
                                                             shuffle=True,
                                                             random_state=RANDOM_SEED )

In [45]:
import featuretools as ft

In [46]:
data = pd.DataFrame(x_train_val, columns=df_orig.columns[:-1])
data['index'] = data.index
es = ft.EntitySet(id='asd')
es = es.entity_from_dataframe(
                              entity_id = 'wines',
                              dataframe = data,
                              index='index'
                             )

features_train, feature_names = ft.dfs(
                                 entityset=es,
                                 target_entity='wines', 
                                 max_depth = 2,
                                 verbose = 1, 
                                 n_jobs = -1,
                                 trans_primitives = ['multiply_numeric'],
                                )

Built 78 features
EntitySet scattered to 12 workers in 5 seconds                                                                         
Elapsed: 00:01 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████


In [47]:
data = pd.DataFrame(x_test, columns=df_orig.columns[:-1])
data['index'] = data.index
es = ft.EntitySet(id='asd')
es = es.entity_from_dataframe(
                              entity_id = 'wines',
                              dataframe = data,
                              index='index'
                             )

features_test, feature_names = ft.dfs(
                                 entityset=es,
                                 target_entity='wines', 
                                 max_depth = 2,
                                 verbose = 1, 
                                 n_jobs = -1,
                                 trans_primitives = ['multiply_numeric'],
                                )

Built 78 features
EntitySet scattered to 12 workers in 5 seconds                                                                         
Elapsed: 00:01 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████


# Logisitic Regression

In [22]:
from sklearn.linear_model import LogisticRegressionCV

In [29]:
res = []

ss = StandardScaler()
x_train_val = ss.fit_transform(features_train)
x_test = ss.transform(features_test)

class_weight = {label:(y_train_val==label).sum()/len(y_train_val) for label in y_train_val.unique()}
lr = LogisticRegressionCV(random_state=RANDOM_SEED,  
                          n_jobs=-1,
                          Cs=3,
                          cv = 5,
                          max_iter=500,
                          class_weight=class_weight,
                         )

lr.fit(x_train_val, y_train_val)

pred_train = lr.predict(x_train_val)
acc_train = (y_train_val == pred_train).sum() / len(y_train_val)
f1_train = f1_score(y_train_val, pred_train, average='weighted')
bal_acc_train = recall_score(y_train_val, pred_train, average='weighted')
res.append(['train', acc_train, f1_train, bal_acc_train, len(y_train_val)])

pred_test = lr.predict(x_test)
acc_test = (y_test == pred_test).sum() / len(y_test)
f1_test = f1_score(y_test, pred_test, average='weighted')
bal_acc_test = recall_score(y_test, pred_test, average='weighted')
res.append(['test', acc_test, f1_test, bal_acc_test, len(y_test)])

df_res = pd.DataFrame(res, columns = ['Eval', 'Acc', 'Rec', 'F1', 'Support'])
df_res

C:\Users\Hades\Anaconda3\envs\torch_cv\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Eval,Acc,Rec,F1,Support
0,train,0.548743,0.488808,0.548743,5170
1,test,0.527456,0.464367,0.527456,1293


In [30]:
pd.DataFrame(np.array(precision_recall_fscore_support(y_test, pred_test, average=None)).T,
             columns = ['Prec', 'Rec', 'F1', 'Support'])

,Prec,Rec,F1,Support
0,1.000000,0.061224,0.115385,49.0
1,0.639394,0.495305,0.558201,426.0
2,0.489926,0.819149,0.613139,564.0
3,0.375000,0.027907,0.051948,215.0
4,0.000000,0.000000,0.000000,39.0


# SVM

In [54]:
from sklearn.model_selection import cross_val_score
from sklearn import svm

In [56]:
df = df_orig.copy()
df['type'] = df.type.map({'red': 1, 'white': 0})
df.drop(columns='type', inplace=True)
df['quality'] = df['quality'].map({3:0, 4:0, 5:1, 6:2, 7:3, 8:4, 9:4})
x_train_val, x_test, y_train_val, y_test = train_test_split(df.drop(columns=['quality']),
                                                            df['quality'],
                                                             stratify=df['quality'],
                                                             test_size=0.2,
                                                             shuffle=True,
                                                             random_state=RANDOM_SEED )

res = []

ss = StandardScaler()
x_train_val = ss.fit_transform(features_train)
x_test = ss.transform(features_test)

clf = svm.SVC(kernel='rbf', C=300, cache_size=4000, random_state=RANDOM_SEED)
clf.fit(x_train_val, y_train_val)

pred_train = clf.predict(x_train_val)
acc_train = (y_train_val == pred_train).sum() / len(y_train_val)
f1_train = f1_score(y_train_val, pred_train, average='weighted')
bal_acc_train = recall_score(y_train_val, pred_train, average='weighted')
res.append(['train', acc_train, f1_train, bal_acc_train, len(y_train_val)])

pred_test = clf.predict(x_test)
acc_test = (y_test == pred_test).sum() / len(y_test)
f1_test = f1_score(y_test, pred_test, average='weighted')
bal_acc_test = recall_score(y_test, pred_test, average='weighted')
res.append(['test', acc_test, f1_test, bal_acc_test, len(y_test)])

df_res = pd.DataFrame(res, columns = ['Eval', 'Acc', 'Rec', 'F1', 'Support'])
df_res

,Eval,Acc,Rec,F1,Support
0,train,0.855899,0.855232,0.855899,5170
1,test,0.583913,0.579232,0.583913,1293


In [57]:
pd.DataFrame(np.array(precision_recall_fscore_support(y_train_val, pred_train, average=None)).T,
             columns = ['Prec', 'Rec', 'F1', 'Support'])

,Prec,Rec,F1,Support
0,0.929348,0.876923,0.902375,195.0
1,0.891596,0.860165,0.875598,1702.0
2,0.817161,0.903369,0.858105,2256.0
3,0.872603,0.741560,0.801762,859.0
4,0.958333,0.727848,0.827338,158.0


In [58]:
pd.DataFrame(np.array(precision_recall_fscore_support(y_test, pred_test, average=None)).T,
             columns = ['Prec', 'Rec', 'F1', 'Support'])

,Prec,Rec,F1,Support
0,0.230769,0.244898,0.237624,49.0
1,0.633028,0.647887,0.640371,426.0
2,0.592949,0.656028,0.622896,564.0
3,0.535032,0.390698,0.451613,215.0
4,0.541667,0.333333,0.412698,39.0


# LightGBM

In [48]:
np.random.seed(RANDOM_SEED)
features_train['weight'] = y_train_val.map(y_train_val.value_counts(normalize=True))
features_train['weight']
light_ds_train = lightgbm.Dataset(features_train, label=y_train_val, weight='weight')
features_train.drop(columns='weight', inplace=True)

In [49]:
import lightgbm

In [50]:
parameters = {'application': 'multiclass',
              'bagging_fraction': 0.75,
              'bagging_freq': 3,
              'boosting': 'gbdt',
              'feature_fraction': 0.85,
              'is_unbalance': 'true',
              'lambda_l1': 1,
              'lambda_l2': 0.01,
              'learning_rate': 0.17,
              'max_depth': 8,
              'metric': 'multi_error',
              'num_class': 5,
              'num_leaves': 14,
             }

In [52]:
model = lightgbm.train(parameters,
                       light_ds_train,
                       num_boost_round=350,
#                        early_stopping_rounds=10,
                      )

res = []
pred_train = np.argmax(model.predict(features_train), axis=1)
acc_train = (y_train_val == pred_train).sum() / len(y_train_val)
f1_train = f1_score(y_train_val, pred_train, average='weighted')
bal_acc_train = recall_score(y_train_val, pred_train, average='weighted')
res.append(['train', acc_train, f1_train, bal_acc_train, len(y_train_val)])

pred_test = np.argmax(model.predict(features_test), axis=1)
acc_test = (y_test == pred_test).sum() / len(y_test)
f1_test = f1_score(y_test, pred_test, average='weighted')
bal_acc_test = recall_score(y_test, pred_test, average='weighted')
res.append(['test', acc_test, f1_test, bal_acc_test, len(y_test)])

df_res = pd.DataFrame(res, columns = ['Eval', 'Acc', 'Rec', 'F1', 'Support'])
df_res

,Eval,Acc,Rec,F1,Support
0,train,0.998839,0.998839,0.998839,5170
1,test,0.668213,0.663360,0.668213,1293


In [53]:
pd.DataFrame(np.array(precision_recall_fscore_support(y_test, pred_test, average=None)).T,
             columns = ['Prec', 'Rec', 'F1', 'Support'])

,Prec,Rec,F1,Support
0,0.640000,0.326531,0.432432,49.0
1,0.712941,0.711268,0.712103,426.0
2,0.647335,0.732270,0.687188,564.0
3,0.632432,0.544186,0.585000,215.0
4,0.750000,0.384615,0.508475,39.0


# best results so far